<a href="https://colab.research.google.com/github/nikomen/CBE_AI/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Podłączenie google drive z Colabem
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

#Wczytanie danych
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/all.csv')

In [3]:
# Miałem problem z IP oraz czasem jako danymi. Na razie je wywaliłem
dataset = dataset.drop(['SourceIP','DestinationIP','PacketTimeMode','TimeStamp'],1)
# Wywalam duplikaty i próbki które nie miały wszystkich cech
dataset = dataset.dropna()
dataset = dataset.drop_duplicates()
dataset

<ipython-input-3-7478579b57e2>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dataset = dataset.drop(['SourceIP','DestinationIP','PacketTimeMode','TimeStamp'],1)


,SourcePort,DestinationPort,Duration,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,...,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation,DoH
0,49972,443,0.017147,171,9.972590e+03,105,6.123520e+03,3.380000e+02,18.384776,92.000000,...,0.707107,0.000000e+00,0.000000,0.017120,0.017120,0.017120,-10.000000,-10.000000,0.000000,True
1,50028,443,0.954806,2555,2.675936e+03,5675,5.943616e+03,3.106133e+05,557.326960,257.187500,...,1.353916,8.367029e-05,0.009147,0.010709,0.008997,0.000004,0.561552,1.170332,0.854139,True
2,50092,443,1.646289,8465,5.141868e+03,10845,6.587543e+03,1.360269e+04,116.630552,132.260274,...,0.549101,6.383296e-05,0.007990,0.010945,0.015319,0.000003,-1.642429,1.369527,0.729979,True
3,60540,443,1.132705,1751,1.545857e+03,4504,3.976322e+03,6.008720e+05,775.159356,347.500000,...,0.847468,1.275703e-03,0.035717,0.079840,0.095209,0.000012,-1.290885,2.235006,0.447359,False
4,50928,443,1.170098,2351,2.009233e+03,4633,3.959497e+03,2.082705e+05,456.366649,258.666667,...,0.806748,1.191502e-03,0.034518,0.054705,0.082358,0.000016,-2.403374,1.584345,0.630991,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371831,443,49440,13.777943,3165,2.297150e+02,2286,1.659174e+02,9.243248e+03,96.141813,126.767442,...,0.317461,5.656498e-08,0.000238,0.000155,0.000050,0.000004,1.330187,0.636808,1.529926,True
371832,35308,443,40.117653,3209,7.998972e+01,47363,1.180602e+03,1.563846e+06,1250.538417,857.152542,...,2.888559,6.483573e+01,8.052064,1.743961,0.008872,0.000046,0.646451,0.216580,4.617112,False
371833,443,49440,177.001521,279,1.576258e+00,720,4.067762e+00,2.851875e+02,16.887495,83.250000,...,0.767390,3.583343e-04,0.018930,0.013419,0.000034,0.000034,2.121320,0.707107,1.410630,True
371834,443,49440,0.000026,93,3.576923e+06,54,2.076923e+06,3.802500e+02,19.500000,73.500000,...,1.000000,0.000000e+00,0.000000,0.000026,0.000026,0.000026,-10.000000,-10.000000,0.000000,True


In [4]:
# Sprawdzenie jak zbalansowane są dane
dataset.groupby(dataset.DoH).size()

DoH
False    353122
True      16039
dtype: int64

In [5]:
# Balansowanie danych
from sklearn.utils import resample
Klasa_mniejsza = dataset[dataset.DoH == True]
Klasa_wieksza = dataset[dataset.DoH == False]
Klasa_wieksza_zmniejszona = resample(Klasa_wieksza, replace=False, n_samples=16039, random_state=42)
dataset = pd.concat([Klasa_wieksza_zmniejszona, Klasa_mniejsza])
dataset.groupby(dataset.DoH).size()

DoH
False    16039
True     16039
dtype: int64

In [6]:
# Dzielę dane na cechy i etykiety
X = dataset.iloc[:, :-1]
# Zamiana wartości etykiet z True/False na 1/0.
y = dataset.iloc[:, -1].astype(int)

# Standaryzacja danych Standaryzacja polega na przekształceniu danych w taki sposób, że średnia wartość (średnia arytmetyczna) wszystkich danych wynosi 0, a odchylenie standardowe wynosi 1.
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler()
X = X_scaled.fit_transform(X)

In [7]:
# Podział na dane treningowe i dane testowe
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [8]:
# Wybór X najlepszych cech
from sklearn.feature_selection import SelectKBest, mutual_info_classif
select_feature = SelectKBest(mutual_info_classif, k=17).fit(X_train,y_train)
X_train = select_feature.transform(X_train)
X_test = select_feature.transform(X_test)

In [9]:
# Wybieram algorytm uczenia maszynowego
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=1442)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [10]:
# Wybieram algorytm uczenia maszynowego
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [15]:
# Wybieram algorytm uczenia maszynowego - Random Forest (RF),
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [12]:
# Wybieram algorytm uczenia maszynowego - support Vector Machines
from sklearn import svm
model = svm.SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [13]:
# Wybieram algorytm uczenia maszynowego - deep neural network DNN
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(alpha=0.00001, hidden_layer_sizes=(5, 2), random_state=1, solver='lbfgs', max_iter=100000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [16]:
# Sprawdzam jak model sobie poradził
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Accuracy - Miara dokładności klasyfikacji.
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Precision - Miara dokładności pozytywnych predykcji (tp / (tp + fp)).
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Recall - Miara pokrycia pozytywnych przypadków (tp / (tp + fn)).
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# F1 Score - Średnia harmoniczna precision i recall.
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

# Confusion Matrix - Macierz błędów, która zawiera informacje o tp, tn, fp i fn.
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion)


Accuracy: 0.9197007481296758
Precision: 0.9104922923918448
Recall: 0.9280283831728332
F1 Score: 0.9191767068273092
Confusion Matrix:
[[3714  360]
 [ 284 3662]]


In [17]:
#Cross-validation score (wynik walidacji krzyżowej) to metoda oceny wydajności modelu uczenia maszynowego, która pomaga w określeniu, jak dobrze model generalizuje na nie widziane wcześniej dane
from sklearn.model_selection import cross_val_score
cv_score = cross_val_score(model, X,y,cv=5,scoring="accuracy")
cv_score.mean()

0.9022032214864243